# Benchmarks

## Initialization and imports

In [ ]:
from sklearn.decomposition import NMF
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

from sknetwork.clustering import BiLouvain
from sknetwork.embedding import RandomProjection, HLouvainEmbedding, SVD
from sknetwork.data import load_netset
from sknetwork.topology import connected_components
from sknetwork.utils.timeout import TimeOut

from nodevectors import ProNE, GGVec

from time import time
import pickle
from itertools import count, filterfalse
from pprint import pprint
import gc

from scipy import sparse
import numpy as np
import pandas as pd

In [ ]:
# Cluster NE implementation
from typing import Optional, Union

import numpy as np
from scipy import sparse

from sknetwork.clustering.louvain import BiLouvain
from sknetwork.embedding.base import BaseBiEmbedding
from sknetwork.linalg.normalization import normalize
from sknetwork.utils.check import check_random_state, check_adjacency_vector, check_nonnegative
from sknetwork.utils.membership import membership_matrix

class ClusterNE(BaseBiEmbedding):
    """Embedding of bipartite graphs induced by Louvain clustering. Each component of the embedding corresponds
    to a cluster obtained by Louvain.

    Parameters
    ----------
    resolution : float
        Resolution parameter.
    modularity : str
        Which objective function to maximize. Can be ``'dugue'``, ``'newman'`` or ``'potts'``.
    tol_optimization :
        Minimum increase in the objective function to enter a new optimization pass.
    tol_aggregation :
        Minimum increase in the objective function to enter a new aggregation pass.
    n_aggregations :
        Maximum number of aggregations.
        A negative value is interpreted as no limit.
    shuffle_nodes :
        Enables node shuffling before optimization.
    random_state :
        Random number generator or random seed. If ``None``, numpy.random is used.
    isolated_nodes : str
        What to do with isolated column nodes. Can be ``'remove'`` (default), ``'merge'`` or ``'keep'``.

    Attributes
    ----------
    embedding_ : array, shape = (n, n_components)
        Embedding of the nodes.
    embedding_row_ : array, shape = (n_row, n_components)
        Embedding of the rows (copy of **embedding_**).
    embedding_col_ : array, shape = (n_col, n_components)
        Embedding of the columns.
    """
    def __init__(self, resolution: float = 1, modularity: str = 'dugue', tol_optimization: float = 1e-3,
                 tol_aggregation: float = 1e-3, n_aggregations: int = -1, shuffle_nodes: bool = False,
                 random_state: Optional[Union[np.random.RandomState, int]] = None, isolated_nodes: str = 'remove'):
        super(ClusterNE, self).__init__()
        self.resolution = np.float32(resolution)
        self.modularity = modularity.lower()
        self.tol_optimization = np.float32(tol_optimization)
        self.tol_aggregation = tol_aggregation
        self.n_aggregations = n_aggregations
        self.shuffle_nodes = shuffle_nodes
        self.random_state = check_random_state(random_state)
        self.isolated_nodes = isolated_nodes

        self.labels_ = None

    def fit(self, biadjacency: sparse.csr_matrix):
        """Embedding of bipartite graphs from the clustering obtained with Louvain.

        Parameters
        ----------
        biadjacency:
            Biadjacency matrix of the graph.

        Returns
        -------
        self: :class:`BiLouvainEmbedding`
        """
        bilouvain = BiLouvain(resolution=self.resolution, modularity=self.modularity,
                              tol_optimization=self.tol_optimization, tol_aggregation=self.tol_aggregation,
                              n_aggregations=self.n_aggregations, shuffle_nodes=self.shuffle_nodes, sort_clusters=False,
                              return_membership=True, return_aggregate=True, random_state=self.random_state)
        bilouvain.fit(biadjacency)

        self.labels_ = bilouvain.labels_

        embedding_row = bilouvain.membership_row_
        embedding_col = bilouvain.membership_col_

        if self.isolated_nodes in ['remove', 'merge']:
            # remove or merge isolated column nodes and reindex labels
            labels_unique, counts = np.unique(bilouvain.labels_col_, return_counts=True)
            n_labels = max(labels_unique) + 1
            labels_old = labels_unique[counts > 1]
            if self.isolated_nodes == 'remove':
                labels_new = -np.ones(n_labels, dtype='int')
            else:
                labels_new = len(labels_old) * np.ones(n_labels, dtype='int')
            labels_new[labels_old] = np.arange(len(labels_old))
            labels_col = labels_new[bilouvain.labels_col_]

            # reindex row labels accordingly
            labels_unique = np.unique(bilouvain.labels_row_)
            n_labels = max(labels_unique) + 1
            labels_new = -np.ones(n_labels, dtype='int')
            labels_new[labels_old] = np.arange(len(labels_old))
            labels_row = labels_new[bilouvain.labels_row_]

            # get embeddings
            probs = normalize(biadjacency)
            embedding_row = probs.dot(membership_matrix(labels_col))
            probs = normalize(biadjacency.T)
            embedding_col = probs.dot(membership_matrix(labels_row))

        self.embedding_row_ = embedding_row.toarray()
        self.embedding_col_ = embedding_col.toarray()
        self.embedding_ = self.embedding_row_

        return self

In [ ]:
# Dataset selection
graph = load_netset('polblogs')

In [ ]:
# Datasets statistics
graph

In [ ]:
# Number of labels
len(set(graph.labels))

In [ ]:
# Auto-dimension setting

cne = ClusterNE()
N_COMPONENTS = cne.fit_transform(graph.adjacency).shape[1]
print(N_COMPONENTS)

In [ ]:
# Algorithms under study
methods = {
           'clusterNE': ClusterNE,
           'NMF' : NMF,
           'SVD' : SVD,
           'randNE': RandomProjection,
           'proNE': ProNE,
           'louvainNE': HLouvainEmbedding,
          }

In [ ]:
# Initializing algorithms
algorithms = {}
embeddings = {}
variant_name = f'Dim. {N_COMPONENTS}/Res. 1'
algorithms[variant_name] = {}
embeddings[variant_name] = {}
for name in methods:
    sample = methods[name]()
    if hasattr(sample, 'n_components'):
        if hasattr(sample, 'mu'):
            algorithms[variant_name][name] = methods[name](n_components=N_COMPONENTS, mu=.1)
        else:
            algorithms[variant_name][name] = methods[name](n_components=N_COMPONENTS)
    if hasattr(sample, 'resolution'):
        algorithms[variant_name][name] = methods[name]()

In [ ]:
times = {}

In [ ]:
# Creating embeddings
for param_name in algorithms:
    times[param_name] = {}
    for name in algorithms[param_name]:
        dep = time()
        embeddings[param_name][name] = algorithms[param_name][name].fit_transform(graph.adjacency)
        times[param_name][name] = time() - dep

In [ ]:
# Execution times
pd.DataFrame(times)

## Link prediction: AUC on negative vs positive sampling

In [ ]:
N_SAMPLES = 10000
samples = np.zeros((2*N_SAMPLES, 3), dtype=int)
degrees = graph.adjacency.dot(np.ones(graph.adjacency.shape[0], dtype=int))
nodes = np.random.choice(np.arange(graph.adjacency.shape[0], dtype=int)[degrees>0], size = N_SAMPLES)
indptr, indices = graph.adjacency.indptr, graph.adjacency.indices

In [ ]:
for index in range(N_SAMPLES):
    node = nodes[index]
    pos_samp = np.random.choice(indices[indptr[node]:indptr[node+1]])
    neg_samp = next(filterfalse(set(indices[indptr[node]:indptr[node+1]]).__contains__, count(1)))
    samples[2*index,:] = node, pos_samp, 1
    samples[2*index + 1,:] = node, neg_samp, 0

In [ ]:
auc_scores = {}

In [ ]:
for param_name in algorithms:
    auc_scores[param_name] = {}
    for name in algorithms[param_name]:
        similarities = cosine_similarity(embeddings[param_name][name][samples[:,0]],
                                         embeddings[param_name][name][samples[:,1]])
        auc_scores[param_name][name] = roc_auc_score(samples[:,2], np.diagonal(similarities))

In [ ]:
pd.DataFrame(auc_scores)

## Classif: compare ground truth against KMeans on the embedding

In [ ]:
knn_scores = {}

In [ ]:
for param_name in algorithms:
    knn_scores[param_name] = {}
    for name in algorithms[param_name]:
        knn_scores[param_name][name] = 0

In [ ]:
N_RUNS = 3
for i in range(N_RUNS):
    print('Run', i)
    # Drawing samples
    N_TRAINING_PER_CLASS = 100
    n_classes = len(set(graph.labels))
    n_tot = graph.adjacency.shape[0] - N_TRAINING_PER_CLASS * n_classes
    samples = np.zeros(N_TRAINING_PER_CLASS * n_classes, dtype=int)
    for cl in range(n_classes):
        mask = (graph.labels == cl)
        samples[cl * N_TRAINING_PER_CLASS: (cl + 1) * N_TRAINING_PER_CLASS] = np.random.choice(np.arange(len(graph.labels))[mask], N_TRAINING_PER_CLASS)
    complement = np.array(list(set(np.arange(graph.adjacency.shape[0])) - set(samples)))
    
    for param_name in algorithms:
        for name in algorithms[param_name]:
            neigh = SVC()
            neigh.fit(embeddings[param_name][name][samples], graph.labels[samples])
            labels = neigh.predict(embeddings[param_name][name][complement])
            knn_scores[param_name][name] += (graph.labels[complement] == labels).sum() / n_tot
            
for param_name in algorithms:
    for name in algorithms[param_name]:
        knn_scores[param_name][name] /= N_RUNS

In [ ]:
pd.DataFrame(knn_scores)

## Time performance

The adjacency contained in `wdc_hyperlink.npz` should be a subgraph of the [Web Data Commons crawl of the web](http://webdatacommons.org/hyperlinkgraph/). It can be obtained by parsing the first few parts (~ 30) of the edgelist by using `sknetwork.data.load_edge_list` and summing the results.

In [ ]:
adj = sparse.load_npz('wdc_hyperlink.npz')

In [ ]:
adj

In [ ]:
times = {}
TIMEOUT = 7200
key = list(algorithms.keys()).pop()

In [ ]:
# Sizes of the subgraph to consider to obtain the number of edges given in the article
sizes = [int(el) for el in [3e6, 6e6, 1.4e7]]
sizes

In [ ]:
for size in sizes:
    # Slicing to the desired size
    small_adj = adj[:int(size), :][:, :int(size)]
    # Keeping the largest connected component only
    res = connected_components(small_adj)
    _, counts = np.unique(res, return_counts=True)
    top = np.argmax(counts)
    small_adj = small_adj[res == top, :][:, res == top]
    print('Sliced:', repr(small_adj))
    # ClusterNE goes first to set the dimension
    dep = time()
    try:
        with TimeOut(TIMEOUT):
            algorithms[key]['clusterNE'].fit_transform(small_adj)
        times['clusterNE'] = time() - dep
        print(key,'-','clusterNE',':',algorithms[key]['clusterNE'].embedding_.shape[1])
        DIM = algorithms[key]['clusterNE'].embedding_.shape[1]
    except TimeoutError:
        times['clusterNE'] = f'Timeout (exceeded {TIMEOUT}s)'
    with open("times-%.1e.p" % size, 'wb') as file:
        pickle.dump(times, file)
    # Free the RAM for the next algo
    algorithms[key]['clusterNE'].embedding_ = None
    gc.collect()
    for name in algorithms[key]:
        print('Running', name)
        if name != 'clusterNE':
            algorithms[key][name].n_components = DIM
            dep = time()
            try:
                with TimeOut(TIMEOUT):
                    algorithms[key][name].fit_transform(small_adj)
                times[name] = time() - dep
            except TimeoutError:
                times[name] = f'Timeout (exceeded {TIMEOUT}s)'
            # Saving the results
            with open("times-%.1e.p" % size, 'wb') as file:
                pickle.dump(times, file)
            # Free the RAM for the next algo
            algorithms[key][name].embedding_ = None
            gc.collect()

In [ ]:
for size in sizes:
    print('\nSize', size)
    with open('times-%.1e.p' % size, 'rb') as file:
        dico = pickle.load(file)
        for entry in dico:
            print(entry, ':', dico[entry])